Day started collecting data Aug 11 2020

### Explanation:

* Inorder to get the matches of a specific season we need to do the following:
1. get the season code (1 request). 
2. get the basic details of all matches (i.e. Fixtures) (1 request for all matches)
3. get the lineups and more information about a match (1 request per match)
4. get the players ratings of each match (1 request per match)

Things to note:
1. We must query the fixtures corrected to a specified timezone
2. We must find a way to update matches that occur while working on the project.
3. Everything must be automated eventually
4. Some entries such as referee, are not updated right after the match, it might be registerd as None until the api updates its data
5. some matches are not updated right away.

In [43]:
import requests
import pprint
import re

import pandas as pd
import numpy as np
import time

In [45]:
# I change it after every commit
apiKey = "49273dd57dmshd0d11cc4a4c7738p160740jsn640210d99aa3"

In [46]:
season2018 = pd.read_csv("Season2018.csv")
season2019 = pd.read_csv("Season2019.csv")
season2020 = pd.read_csv("Season2020.csv")


Getting the codes for the seasons 2018, 19, 20

In [18]:
#The response contains information of all leagues in a specific country
url = "https://api-football-v1.p.rapidapi.com/v2/leagues/country/SA"

headers = {
    'x-rapidapi-host': "api-football-v1.p.rapidapi.com",
    'x-rapidapi-key': apiKey
    }

response = requests.request("GET", url, headers=headers)

#I'll extract the codes manually

In [19]:
pprint.pprint(response.json())

{'api': {'leagues': [{'country': 'Saudi-Arabia',
                      'country_code': 'SA',
                      'coverage': {'fixtures': {'events': True,
                                                'lineups': True,
                                                'players_statistics': True,
                                                'statistics': True},
                                   'odds': False,
                                   'players': True,
                                   'predictions': True,
                                   'standings': True,
                                   'topScorers': True},
                      'flag': 'https://media.api-sports.io/flags/sa.svg',
                      'is_current': 0,
                      'league_id': 419,
                      'logo': 'https://media.api-sports.io/football/leagues/307.png',
                      'name': 'Pro League',
                      'season': 2018,
                      'season_end': '2019-05

In [7]:
season2018code = 419
season2019code = 901
season2020code = 2984


getting the code for ٍRiyadh's timezone so that the date variable is relative

In [24]:
url = "https://api-football-v1.p.rapidapi.com/v2/timezone"

headers = {
    'x-rapidapi-host': "api-football-v1.p.rapidapi.com",
    'x-rapidapi-key': apiKey
    }

response = requests.request("GET", url, headers=headers)

pprint.pprint(response.json())



{'api': {'results': 425,
         'timezone': ['Africa/Abidjan',
                      'Africa/Accra',
                      'Africa/Addis_Ababa',
                      'Africa/Algiers',
                      'Africa/Asmara',
                      'Africa/Bamako',
                      'Africa/Bangui',
                      'Africa/Banjul',
                      'Africa/Bissau',
                      'Africa/Blantyre',
                      'Africa/Brazzaville',
                      'Africa/Bujumbura',
                      'Africa/Cairo',
                      'Africa/Casablanca',
                      'Africa/Ceuta',
                      'Africa/Conakry',
                      'Africa/Dakar',
                      'Africa/Dar_es_Salaam',
                      'Africa/Djibouti',
                      'Africa/Douala',
                      'Africa/El_Aaiun',
                      'Africa/Freetown',
                      'Africa/Gaborone',
                      'Africa/Harare',
      

requesting the matches data corrected to our timezone

In [8]:
def get_fixtures(seasonCode):
    """Returns all season fixtures as a json type object.

    Args:
    SeasonCode: int or String. Which is the code specified by the
                    api for querying the details of a specific season.


    Returns:
    fixtures: which is a json object that contains all fixtures of the desired season.
    """
    url = f"https://api-football-v1.p.rapidapi.com/v2/fixtures/league/{seasonCode}?timezone=Asia/Riyadh"

    headers = {
        'x-rapidapi-host': "api-football-v1.p.rapidapi.com",
        'x-rapidapi-key': apiKey
        }

    fixtures = requests.request("GET", url, headers=headers)

    return fixtures.json()["api"]["fixtures"]

In [78]:
def create_df(seasonCode, year):
    """Creates a dataframe with the the basic details of the match.

    Args:
    seasonCode:  int or String. Which is the code specified by the
                    api for querying the details of a specific season.
    
    year: String. Which specifies which season are we creating the dataframe for (e.g. 2020/2021)

    Returns:
    
    season_df: Pandas Dataframe. that is consisted of all matches 'basic' details.
    
    **** Note: for creating a dataframe for a full completed season this might****
    """
    
    matches = get_fixtures(seasonCode)
    
    season_df = pd.DataFrame(columns=['fixture_id', 'home_team', 'away_team', 'home_goals',
                                      'away_goals', 'venue', 'referee',
                                      'round', 'date', 'season','status',
                                     "home_coach", "home_formation", "home_players",
                                    "away_coach", "away_formation", "away_players",
                                     "H_avg_ratings", "A_avg_ratings"])
    
    for i in range(240):
        season_df = addMatchInfo(season_df, year, i, matches[i])
        season_df = addMatchLineups(seaosn_df, i)
        season_df = addPlayerRatings(season_df, i)
            
    return season_df

In [11]:
def addMatchInfo(season_df, year, match_index, match_info):
    
    fixture_id = match_info["fixture_id"]
    home_team = match_info["homeTeam"]["team_name"]
    away_team = match_info["awayTeam"]["team_name"]
    referee = match_info["referee"]
    nRound = match_info["round"].split("-")[1].replace(" ", "")
    date = match_info["event_date"]
    venue = match_info["venue"]
    home_goals = match_info["goalsHomeTeam"]
    away_goals = match_info["goalsAwayTeam"]
    status = match_info["status"]

    season_df.loc[match_index] = [fixture_id, home_team,
                         away_team, home_goals,
                         away_goals, venue,
                         referee, nRound,
                         date, year, status,
                        np.nan, np.nan, np.nan, #These columns require a request per match,
                        np.nan, np.nan, np.nan,
                                 np.nan, np.nan]
    return season_df

In [50]:
def addMatchLineups(season_df, match_index):
    """Adds all the extra information of finished matches to the dataframe.

    Args:
    season_df: Pandas Dataframe. that is consisted of all matches 'basic' details.
    
    match_index: index of the match to be updated


    Returns:
    season_df: Pandas Dataframe. The same dataframe provided but with added 
                                features to the finished matches
    """
    

    match = season_df.loc[match_index]
    
    url = f"https://api-football-v1.p.rapidapi.com/v2/lineups/{match['fixture_id']}"

    headers = {
        'x-rapidapi-host': "api-football-v1.p.rapidapi.com",
        'x-rapidapi-key': apiKey
        }

    response = requests.request("GET", url, headers=headers).json()

    #home team data

    home_coach = response["api"]["lineUps"][match["home_team"]]["coach"]
    home_formation = response["api"]["lineUps"][match["home_team"]]["formation"]

    home_players = []
    for player in response["api"]["lineUps"][match["home_team"]]["startXI"]:
        home_players.append((player["player"], player["pos"]))


    #away team data                                             
    away_coach = response["api"]["lineUps"][match["away_team"]]["coach"]
    away_formation = response["api"]["lineUps"][match["away_team"]]["formation"]

    away_players = []
    for player in response["api"]["lineUps"][match["away_team"]]["startXI"]:
        away_players.append((player["player"], player["pos"]))                                            

    season_df.loc[match_index, ["home_coach", "home_formation",
                  "home_players", "away_coach",
                  "away_formation", "away_players"]] = [home_coach, home_formation, 
                                                       home_players, away_coach,
                                                       away_formation, away_players]
    
    return season_df

In [13]:
def addPlayerRatings(season_df, match_index):
    
    fixture_id = season_df.loc[match_index, "fixture_id"]
    home_team = season_df.loc[match_index, "home_team"]
    
    url = f"https://api-football-v1.p.rapidapi.com/v2/players/fixture/{fixture_id}"

    headers = {
        'x-rapidapi-host': "api-football-v1.p.rapidapi.com",
        'x-rapidapi-key': apiKey
        }

    response = requests.request("GET", url, headers=headers)

    response.json()
    
    players = response.json()["api"]["players"]

    Hteam_ratings = []
    Ateam_ratings = []
    
    for player in players:

        try: #Incase the rating is null, we can't cast it
            if player["team_name"] == home_team:
                Hteam_ratings.append(float(player["rating"]))
            else:
                Ateam_ratings.append(float(player["rating"]))    
        except:
            pass

    Hteam_avg_ratings = round(sum(Hteam_ratings) / len(Hteam_ratings), 2)
    Ateam_avg_ratings = round(sum(Ateam_ratings) / len(Ateam_ratings), 2)
        
    season_df.loc[match_index, ["H_avg_ratings", "A_avg_ratings"]] = [Hteam_avg_ratings, Ateam_avg_ratings]
    
    return season_df


In [14]:
def update_match(match, season_df):
    
    match_index = season_df.loc[season_df["fixture_id"] == match["fixture_id"]].index[0]
    
    season_df = addMatchInfo(season_df, season_df.season.unique()[0], match_index, match)

    season_df = addMatchLineups(season_df, match_index)
    
    season_df = addPlayerRatings(season_df, match_index)
    
    return season_df

In [22]:
#The following code was used to extract matches that happend while I was working on the project
def update_df(season_df, season_code):

    not_started_matches = season_df.query("status == 'Not Started'").index

    all_matches = get_fixtures(season_code)
    
    for i in not_started_matches:
        match = all_matches[i]
        if match["status"] == "Match Finished":
            season_df = update_match(match, season_df)
        
    return season_df


-----

In [78]:
season2020_fixtures = get_fixtures(season2020code)

Now we should have the data for all matches in the 2018/2019 season.
We have 16 teams playing 30 rounds so we got 16 * 30 / 2 = 240 matches

In [15]:
len(season2020_fixtures)

240

Checking how each match is structured

In [69]:
season2020 = create_df(season2020code, "2020/2021")

In [149]:
season2020.head(30)

,fixture_id,home_team,away_team,home_goals,away_goals,venue,referee,round,date,season,status,home_coach,home_foramtion,home_players,away_coach,away_formation,away_players,H_avg_ratings,A_avg_ratings
0,635867,Al Shabab,Abha,1.0,0.0,Prince Faisal bin Fahd Stadium,Mohammed Al Hoaish,1,2020-10-17T18:00:00+03:00,2020/2021,Match Finished,NaN,4-4-2,"[('Giedrius Arlauskis', 'G'), ('Mohammed Salem...",A. Chebbi,4-2-3-1,"[('Abdelali Mhamdi', 'G'), ('Saeed Ali', 'D'),...",6.51,7.24
1,635868,Al Taawon,Al-Faisaly FC,1.0,1.0,King Abdullah bin Abdulaziz City,Faisal Alblwi,1,2020-10-17T18:10:00+03:00,2020/2021,Match Finished,Abdullah Assiri,5-4-1,"[('Cássio', 'G'), ('Faisal Darwish', 'D'), ('A...",Péricles Chamusca,4-3-3,"[('Mustafa Malaika', 'G'), ('Khaled Daghriri',...",NaN,NaN
2,635869,Al Wehda Club,Al-Qadisiyah FC,2.0,1.0,King Abdulaziz Sports City Stadium,Khaled Salwi,1,2020-10-17T20:00:00+03:00,2020/2021,Match Finished,NaN,4-4-2,"[('Abdullah Hussein Al Owisher', 'G'), ('Hamad...",NaN,4-3-2-1,"[('Faisal Al-Masrahi', 'G'), ('Hamad Al Tohaif...",NaN,NaN
3,635870,Al-Hilal Saudi FC,Al-Ain,1.0,0.0,Prince Faisal bin Fahd Stadium,Sultan Al-Harbi,1,2020-10-17T21:15:00+03:00,2020/2021,Match Finished,NaN,4-4-2,"[('Abdullah Al-Mayouf', 'G'), ('Amiri Kurdi', ...",NaN,4-2-3-1,"[('Mohammed Mazyad Alshammari', 'G'), ('Hassan...",NaN,NaN
4,635871,Al Baten,Al-Ahli Jeddah,0.0,1.0,Al-Batin FC Stadium,Khaled Al Teris,1,2020-10-18T18:00:00+03:00,2020/2021,Match Finished,José Garrido,4-3-2-1,"[('Mazyad Al Enazi', 'G'), ('Saad Al Khayri', ...",V. Milojević,4-2-3-1,"[('Yasser Al-Mosailem', 'G'), ('Hassoun', 'D')...",NaN,NaN
5,635872,Al-Nassr,Al-Fateh,1.0,2.0,Prince Faisal bin Fahd Stadium,Majed Al Shamrani,1,2020-10-18T18:00:00+03:00,2020/2021,Match Finished,Rui Vitória,4-2-3-1,"[('Brad Jones', 'G'), ('Osama Al Khalaf', 'D')...",Y. Ferrera,4-4-2,"[('Maksym Koval', 'G'), ('Nawaf Boushal', 'D')...",NaN,NaN
6,635873,Dhamk,Al-Raed,2.0,3.0,Prince Sultan bin Abdulaziz Sports City Stadium,Shukri Al Hanfosh,1,2020-10-18T18:20:00+03:00,2020/2021,Match Finished,N. Zekri,4-2-3-1,"[('Moustapha Zeghba', 'G'), ('Tariq Mohammed',...",B. Hasi,4-3-3,"[('Ahmad Al Harbi', 'G'), ('Mohammed Al-Dosari...",NaN,NaN
7,635874,Al-Ittihad FC,Al-Ettifaq,1.0,2.0,King Abdullah Sports City,Turki Al Khudair,1,2020-10-18T20:00:00+03:00,2020/2021,Match Finished,Fábio Carille,4-4-2,"[('Marcelo Grohe', 'G'), ('Saud Abdulhamid', '...",Khaled Al Atawi,4-4-2,"[(""Raïs M'Bolhi"", 'G'), ('Saleh Al Qumayzi', '...",NaN,NaN
8,635875,Abha,Al-Hilal Saudi FC,1.0,1.0,Prince Sultan bin Abdulaziz Sports City Stadium,Majed Al Shamrani,2,2020-10-22T15:40:00+03:00,2020/2021,Match Finished,A. Chebbi,4-2-3-1,"[('Abdelali Mhamdi', 'G'), ('Saeed Al Hamsal',...",R. Lucescu,4-2-3-1,"[('Abdullah Al-Mayouf', 'G'), ('Mohammed Al-Bu...",NaN,NaN
9,635876,Al Taawon,Al-Nassr,1.0,0.0,King Abdullah bin Abdulaziz City,Mohammed Al Hoaish,2,2020-10-22T18:05:00+03:00,2020/2021,Match Finished,Abdullah Assiri,4-2-3-1,"[('Cássio', 'G'), ('Yaseen Barnawi', 'D'), ('A...",Rui Vitória,4-2-3-1,"[('Brad Jones', 'G'), ('Jin-su Kim', 'D'), ('A...",NaN,NaN


In [61]:
season2020 = update_df(season2020, season2020code)

In [52]:
season2020.loc[season2020["status"] == "Not Started"]

,fixture_id,home_team,away_team,home_goals,away_goals,venue,referee,round,date,season,status,home_coach,home_formation,home_players,away_coach,away_formation,away_players,H_avg_ratings,A_avg_ratings
214,657857,Al-Raed,Al Taawon,NaN,NaN,King Abdullah bin Abdulaziz City,NaN,27,2021-05-15T19:20:00+03:00,2020/2021,Not Started,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
220,657863,Abha,Al-Ittihad FC,NaN,NaN,Prince Sultan bin Abdulaziz Sports City Stadium,NaN,28,2021-05-20T19:15:00+03:00,2020/2021,Not Started,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
222,657865,Al Wehda Club,Dhamk,NaN,NaN,King Abdulaziz Sports City Stadium,NaN,28,2021-05-20T19:30:00+03:00,2020/2021,Not Started,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
223,657866,Al-Nassr,Al-Raed,NaN,NaN,King Saud University Stadium,NaN,28,2021-05-20T21:00:00+03:00,2020/2021,Not Started,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
224,657867,Al-Faisaly FC,Al Shabab,NaN,NaN,Al Majma'ah Sports City Stadium,NaN,29,2021-05-25T21:00:00+03:00,2020/2021,Not Started,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
225,657868,Abha,Al-Ahli Jeddah,NaN,NaN,Prince Sultan bin Abdulaziz Sports City Stadium,NaN,29,2021-05-25T21:00:00+03:00,2020/2021,Not Started,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
226,657869,Al-Ittihad FC,Al-Ain,NaN,NaN,King Abdullah Sports City,NaN,29,2021-05-25T21:00:00+03:00,2020/2021,Not Started,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
227,657870,Al Baten,Dhamk,NaN,NaN,Al-Batin FC Stadium,NaN,29,2021-05-25T21:00:00+03:00,2020/2021,Not Started,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
228,657871,Al-Ettifaq,Al-Raed,NaN,NaN,Prince Mohamed bin Fahd Stadium,NaN,29,2021-05-25T21:00:00+03:00,2020/2021,Not Started,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
229,657872,Al-Fateh,Al-Qadisiyah FC,NaN,NaN,Prince Abdullah bin Jalawi Sports City Stadium,NaN,29,2021-05-25T21:00:00+03:00,2020/2021,Not Started,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [69]:
season2020.loc[(season2020["H_avg_ratings"].isnull() == True) & (season2020["status"] == "Match Finished")]

,fixture_id,home_team,away_team,home_goals,away_goals,venue,referee,round,date,season,status,home_coach,home_formation,home_players,away_coach,away_formation,away_players,H_avg_ratings,A_avg_ratings


In [75]:
season2020.to_csv("Season2020.csv", index=False)